In [7]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any
from datetime import datetime
from collections import defaultdict

class EnhancedWalletProfiler:
    def __init__(self, data_loader):
        self.data_loader = data_loader
        self.global_metrics = {}
        self.wallet_profiles = {}
        
    def calculate_global_metrics(self):
        """Розрахунок глобальних метрик тільки для не-контрактних адрес"""
        if self.data_loader.nodes_data is None:
            return
            
        # Отримуємо тільки не-контрактні адреси
        non_contract_addresses = self.data_loader.get_non_contract_addresses()
        nodes_df = self.data_loader.nodes_data[
            self.data_loader.nodes_data['address'].isin(non_contract_addresses)
        ]
        
        all_txs = []
        for txs in nodes_df['txs']:
            all_txs.extend(txs)
            
        if all_txs:
            tx_values = [float(tx['value$']) for tx in all_txs]
            
            self.global_metrics = {
                'total_wallets': len(non_contract_addresses),
                'total_transactions': len(all_txs),
                'avg_tx_value': np.mean(tx_values),
                'median_tx_value': np.median(tx_values),
                'avg_txs_per_wallet': len(all_txs) / len(non_contract_addresses),
                'total_volume': sum(tx_values)
            }
            
            # Часові метрики
            all_dates = []
            for _, row in nodes_df.iterrows():
                all_dates.append(row['first_active'])
                all_dates.append(row['last_active'])
                
            self.global_metrics.update({
                'network_start_date': min(all_dates),
                'network_last_date': max(all_dates),
                'avg_wallet_lifetime': nodes_df['active_days'].mean()
            })
            
    def create_wallet_profile(self, address: str) -> Dict[str, Any]:
        """Створення профілю гаманця"""
        # Перевіряємо чи це контракт
        if self.data_loader.is_contract(address):
            return self._create_contract_profile(address)
            
        return self._create_full_profile(address)
        
    def _create_contract_profile(self, address: str) -> Dict[str, Any]:
        """Створення базового профілю для контракту"""
        node_data = self.data_loader.get_node_data(address)
        if node_data is None:
            return {}
            
        transactions = self.data_loader.get_node_transactions(address)
        unique_interactors = set()
        total_volume = 0
        
        for tx in transactions:
            unique_interactors.add(tx['from'])
            unique_interactors.add(tx['to'])
            total_volume += float(tx['value$'])
            
        return {
            'address': address,
            'is_contract': True,
            'basic_info': {
                'label': self.data_loader.get_address_info(address).get('label', 'Unknown'),
                'creation_date': node_data['first_active'],
                'last_interaction': node_data['last_active'],
                'total_interactions': len(transactions),
                'unique_interactors': len(unique_interactors),
                'total_volume': total_volume
            }
        }
        
    def _create_full_profile(self, address: str) -> Dict[str, Any]:
        """Створення повного профілю для звичайної адреси"""
        if address in self.wallet_profiles:
            return self.wallet_profiles[address]
            
        transactions = self.data_loader.get_node_transactions(address)
        if not transactions:
            return {}
            
        # Базові метрики
        sent_txs = [tx for tx in transactions if tx['from'] == address]
        received_txs = [tx for tx in transactions if tx['to'] == address]
        
        # Контрактні взаємодії
        contract_interactions = self.data_loader.get_contract_interactions(address)
        
        # Аналіз транзакцій
        total_sent = sum(float(tx['value$']) for tx in sent_txs)
        total_received = sum(float(tx['value$']) for tx in received_txs)
        
        # Створення профілю
        node_data = self.data_loader.get_node_data(address)
        address_info = self.data_loader.get_address_info(address)
        
        profile = {
            'address': address,
            'is_contract': False,
            'label': address_info.get('label', 'Unknown'),
            'has_kyc': address_info.get('hasKyc', False),
            'transactions': {
                'total_count': len(transactions),
                'sent_count': len(sent_txs),
                'received_count': len(received_txs),
                'total_sent': total_sent,
                'total_received': total_received,
                'net_position': total_received - total_sent
            },
            'activity': {
                'first_seen': node_data['first_active'],
                'last_seen': node_data['last_active'],
                'active_days': node_data['active_days'],
                'avg_transactions_per_day': len(transactions) / node_data['active_days'] 
                    if node_data['active_days'] > 0 else 0
            },
            'contract_interactions': {
                'total_contracts': len(contract_interactions),
                'interaction_counts': contract_interactions
            },
            'balances': node_data['balances']
        }
        
        # Порівняння з глобальними метриками
        if self.global_metrics:
            profile['comparison'] = self._calculate_metric_percentiles(profile)
            
        self.wallet_profiles[address] = profile
        return profile
        
    def _calculate_metric_percentiles(self, profile: Dict) -> Dict:
        """Розрахунок перцентилей для метрик гаманця"""
        return {
            'transaction_count_percentile': 
                profile['transactions']['total_count'] / self.global_metrics['avg_txs_per_wallet'],
            'volume_percentile':
                profile['transactions']['total_sent'] / self.global_metrics['total_volume'],
            'lifetime_percentile':
                profile['activity']['active_days'] / self.global_metrics['avg_wallet_lifetime']
        }
        
    def get_wallet_summary(self, address: str) -> Dict[str, Any]:
        """Отримання короткого підсумку по гаманцю"""
        profile = self.create_wallet_profile(address)
        
        if profile.get('is_contract'):
            return {
                'address': address,
                'type': 'Contract',
                'interactions': profile['basic_info']['total_interactions'],
                'volume': profile['basic_info']['total_volume']
            }
            
        return {
            'address': address,
            'type': 'Wallet',
            'transactions': profile['transactions']['total_count'],
            'net_position': profile['transactions']['net_position'],
            'active_days': profile['activity']['active_days'],
            'contract_interactions': profile['contract_interactions']['total_contracts']
        }

    def get_detailed_statistics(self, address: str) -> Dict[str, Any]:
        """Отримання детальної статистики по гаманцю"""
        profile = self.create_wallet_profile(address)
        
        if profile.get('is_contract'):
            return profile['basic_info']
            
        # Розрахунок додаткових статистик для звичайних гаманців
        transactions = self.data_loader.get_node_transactions(address)
        
        # Часові паттерни
        timestamps = [datetime.strptime(tx['timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ') 
                     for tx in transactions]
        
        hour_distribution = defaultdict(int)
        for ts in timestamps:
            hour_distribution[ts.hour] += 1
            
        return {
            'general': {
                'address': address,
                'label': profile['label'],
                'kyc_status': profile['has_kyc']
            },
            'activity': profile['activity'],
            'transactions': profile['transactions'],
            'temporal_patterns': {
                'hour_distribution': dict(hour_distribution),
                'total_active_hours': len(hour_distribution)
            },
            'contract_activity': profile['contract_interactions'],
            'comparative_metrics': profile.get('comparison', {})
        }

In [12]:
%run data_loader.ipynb

import json
from datetime import datetime

def format_timestamp(timestamp):
    """Форматування часової мітки для кращого відображення"""
    dt = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%fZ')
    return dt.strftime('%Y-%m-%d %H:%M:%S')

def test():
    # Ініціалізація
    loader = DataLoader()
    loader.load_addresses("data/addresses.json")
    loader.load_tokens("data/tokens.json")
    loader.load_nodes("data/nodes_new.csv")
    loader.process_node_dates()
    
    profiler = EnhancedWalletProfiler(loader)
    profiler.calculate_global_metrics()
    
    # Тестові адреси (контракт і звичайний гаманець)
    addresses = [
        '0xeba88149813bec1cccccfdb0dacefaaa5de94cb1', # binance
        '0x39cf2e49ea4d620e77d67088a8d815348e0abdf6', # normal
        '0xa1b1bbb8070df2450810b8eb2425d543cfcef79b', # fund
        # '0xa9d1e08c7793af67e9d92fe308d5697fb81d3e43' # contract
    ]

    
    # Аналіз глобальних метрик
    print("\nGlobal Network Metrics:")
    print(json.dumps(profiler.global_metrics, indent=2, default=str))
    

    for address in addresses:
        print(f"\nAnalyzing Wallet: {address}")
        wallet_profile = profiler.create_wallet_profile(address)
        print("\nWallet Profile:")
        print(json.dumps(wallet_profile, indent=2, default=str))
    
        # Короткі підсумки
        print("\nWallet Summary:")
        summary = profiler.get_wallet_summary(address)
        print(f"\nSummary for {address}:")
        print(json.dumps(summary, indent=2, default=str))
    
        # Детальна статистика для гаманця
        print(f"\nDetailed Statistics for wallet {address}:")
        detailed_stats = profiler.get_detailed_statistics(address)
    
        # Форматований вивід деяких ключових метрик
        print("\nKey Metrics:")
        print(f"Total Transactions: {detailed_stats['transactions']['total_count']}")
        print(f"Net Position: {detailed_stats['transactions']['net_position']:.2f}")
        print(f"Active Days: {detailed_stats['activity']['active_days']:.1f}")
    
        if 'temporal_patterns' in detailed_stats:
            print("\nActivity Hours Distribution:")
            hour_dist = detailed_stats['temporal_patterns']['hour_distribution']
            for hour in sorted(hour_dist.keys()):
                print(f"Hour {hour:02d}:00 - {hour_dist[hour]} transactions")
    
        # 6. Порівняльний аналіз
        if 'comparative_metrics' in detailed_stats:
            print("\nComparative Analysis:")
            comp_metrics = detailed_stats['comparative_metrics']
            print(f"Transaction Count vs Average: {comp_metrics.get('transaction_count_percentile', 0):.2f}")
            print(f"Volume vs Network Total: {comp_metrics.get('volume_percentile', 0):.2f}")
            print(f"Lifetime vs Average: {comp_metrics.get('lifetime_percentile', 0):.2f}")
    
        # 7. Контрактні взаємодії
        if detailed_stats.get('contract_activity'):
            print("\nContract Interactions:")
            contract_activity = detailed_stats['contract_activity']
            print(f"Total Unique Contracts: {contract_activity['total_contracts']}")
        
            print("\nTop Contract Interactions:")
            sorted_interactions = sorted(
                contract_activity['interaction_counts'].items(),
                key=lambda x: x[1],
                reverse=True
            )[:5]
        
            for contract, count in sorted_interactions:
                print(f"Contract: {contract}")
                print(f"Interactions: {count}")
                print("---")

In [13]:
test()

Loaded 1733 addresses
Loaded 5825 tokens
Loaded 1025 nodes
Contracts: 51
Non-contracts: 974

Global Network Metrics:
{
  "total_wallets": 974,
  "total_transactions": 522840,
  "avg_tx_value": 54423.73375438928,
  "median_tx_value": 381.27528913721625,
  "avg_txs_per_wallet": 536.7967145790554,
  "total_volume": 28454904956.144886,
  "network_start_date": "2015-08-10 02:39:12",
  "network_last_date": "2024-11-27 08:11:23",
  "avg_wallet_lifetime": 772.3029116044946
}

Analyzing Wallet: 0xeba88149813bec1cccccfdb0dacefaaa5de94cb1

Wallet Profile:
{
  "address": "0xeba88149813bec1cccccfdb0dacefaaa5de94cb1",
  "is_contract": false,
  "label": "Binance",
  "has_kyc": true,
  "transactions": {
    "total_count": 4998,
    "sent_count": 2299,
    "received_count": 2699,
    "total_sent": 2605351.18865,
    "total_received": 2795688.9821857344,
    "net_position": 190337.79353573453
  },
  "activity": {
    "first_seen": "2024-08-29 10:07:35",
    "last_seen": "2024-11-26 14:20:23",
    "activ